In [4]:
import os
import sys
import time

import cv2
import numpy as np
from PIL import Image

import PySpin
import pygame
from pygame.locals import *

import capture_utils as cap

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Total number of buffers
NUM_BUFFERS = 3
SAVE_MODE = False

serial_main = '20356250'
serial_side = '24049908'
# serial_side = '22371101'
serial_list = [serial_main, serial_side]

# Set camera parameters
GAIN = 0
roi = [612, 512, 1224, 1024]
pixel_format = 'BayerRGPolarized8'



pygame 2.1.2 (SDL 2.0.18, Python 3.7.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
def end_acquisition(system, cam, cam_list):
    for i, cam in enumerate(cam_list):
        cam.EndAcquisition()
        cap.reset_trigger(cam)
        cam.DeInit()
        del cam
    cam_list.Clear()
    system.ReleaseInstance()
    pygame.quit()

def main(SHUTTER_TIME, dir, pattern_path, OLAT=False):

    # Set the frames per second
    FPS = 150

    # Set the size of the square
    square_size = 120*2
    square_x = 0
    square_y = 0
    # square_x = 1000
    # square_y = 1000

    # Set the number of frames to display black patterns after each sequence
    BLACK_PATTERN_FRAMES = 24

    # Set the colors
    WHITE = (255, 255, 255)
    GRAY = (255//2, 255//2, 255//2)
    BLACK = (0, 0, 0)
    MAGENTA = (255, 0, 255)
    RED = (255, 0, 0)
    CYAN = (0, 255, 255)
    GREEN = (0, 255, 0)

    WAIT = 10
    
    if not os.path.exists(dir):
        os.mkdir(dir)
        os.mkdir(os.path.join(dir, 'main'))
        os.mkdir(os.path.join(dir, 'side'))

    system, cam, cam_list = cap.initialize_cam(pixel_format, SHUTTER_TIME, roi)

    # Initialize Pygame
    pygame.init()

    # Get the monitor dimensions
    monitor_info = pygame.display.Info()

    # Set the window dimensions to the size of the monitor
    WINDOW_WIDTH = monitor_info.current_w
    WINDOW_HEIGHT = monitor_info.current_h

    # Set the smaller window dimensions
    SMALL_WINDOW_WIDTH = int(WINDOW_WIDTH / 2)
    SMALL_WINDOW_HEIGHT = int(WINDOW_HEIGHT / 2)

    # Set the window caption
    pygame.display.set_caption('Pattern')

    # Create a Pygame clock object
    clock = pygame.time.Clock()

    screen_mode = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT), FULLSCREEN, display=0)

    time.sleep(WAIT)
    num_frame = 0

    if OLAT==False:
        pattern_list = [f for f in os.listdir(pattern_path) if f.endswith('.npy')]
        patterns = []
        pattern_file_names = []
        for idx, pattern_name in enumerate(pattern_list):
            print(pattern_name)
            pattern = np.load(os.path.join(pattern_path, pattern_name)) 
            pattern_ = np.zeros_like(pattern)
            pattern_[:,:,:,0] = pattern[:,:,:,0]**(2.2/1.9301)
            pattern_[:,:,:,1] = pattern[:,:,:,1]**(2.2/1.9378)
            pattern_[:,:,:,2] = pattern[:,:,:,2]**(2.2/1.9462)
            pattern = (pattern_*255).astype(np.uint8)
            for i in range(pattern.shape[0]):
                pattern_ = cv2.resize(pattern[i],(3840, 2160), interpolation=cv2.INTER_NEAREST)
                pattern_ = pattern_.transpose(1,0,2)
                pattern_ = np.flip(pattern_, axis=0)
                patterns.append(pygame.surfarray.make_surface(pattern_))
                pattern_file_names.append(f'{pattern_name.split(".")[0]}_{i}')
        pattern_idx = 0
        pattern_len = len(patterns)
        print(pattern_len)


    screen_mode.fill(WHITE)
    pygame.display.update()
    time.sleep(0.01)
    cap.capture_multiple_cameras(cam_list, dir, f'white_{SHUTTER_TIME}.png')
    
    screen_mode.fill(BLACK)
    pygame.display.update()
    time.sleep(0.01)
    cap.capture_multiple_cameras(cam_list, dir, f'black_{SHUTTER_TIME}.png')
    
    while True:
        # Set the frames per second
        clock.tick(FPS)
        
        for event in pygame.event.get():
            # Handle quitting the game 
            if (event.type == pygame.QUIT) or (event.type == KEYDOWN and event.key == K_q):
                for i, cam in enumerate(cam_list):
                    cam.EndAcquisition()
                    cap.reset_trigger(cam)
                    cam.DeInit()
                    del cam
                cam_list.Clear()
                system.ReleaseInstance()
                pygame.quit()
                sys.exit()
                
            # Toggle fullscreen mode when 'f' is pressed
            elif event.type == KEYDOWN and event.key == K_f:
                if screen_mode.get_flags() & FULLSCREEN:
                    screen_mode = pygame.display.set_mode((SMALL_WINDOW_WIDTH, SMALL_WINDOW_HEIGHT))
                else:
                    screen_mode = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT), FULLSCREEN)
                    
            # Begin buffering images when 'b' is pressed
            elif event.type == KEYDOWN and event.key == K_b:
                BUFFER_MODE = True
                        
        if OLAT:
            if square_y >= WINDOW_HEIGHT:
                for i, cam in enumerate(cam_list):
                    cam.EndAcquisition()
                    cap.reset_trigger(cam)
                    cam.DeInit()
                    del cam
                cam_list.Clear()
                system.ReleaseInstance()
                pygame.quit()
                break
        
            # Fill the surface with a color
            screen_mode.fill(BLACK)        

            # Draw the white square
            pygame.draw.rect(screen_mode, WHITE, (square_x, square_y, square_size, square_size))

            # Update the location of the square
            square_x += square_size
            if square_x >= WINDOW_WIDTH:
                square_x = 0
                square_y += square_size

            # Update the screen
            pygame.display.update()
            cap.capture_multiple_cameras(cam_list, dir, f'{str(num_frame).zfill(3)}_{SHUTTER_TIME}.png')
            num_frame += 1
            
        else:
            if pattern_idx >= pattern_len:
                for i, cam in enumerate(cam_list):
                    cam.EndAcquisition()
                    cap.reset_trigger(cam)
                    cam.DeInit()
                    del cam
                cam_list.Clear()
                system.ReleaseInstance()
                pygame.quit()
                break
            screen_mode.fill(BLACK)        
            screen_mode.blit(patterns[pattern_idx], (0,0))
            pygame.display.update()
            cap.capture_multiple_cameras(cam_list, dir, f'{pattern_file_names[pattern_idx]}_{SHUTTER_TIME}.png')
            pattern_idx+=1
            
        # # TEMPORAL DEBUGGING POINT!!!!!!!!!!!!!!!!!!
        # for i, cam in enumerate(cam_list):
        #     cam.EndAcquisition()
        #     cap.reset_trigger(cam)
        #     cam.DeInit()
        #     del cam
        # cam_list.Clear()
        # system.ReleaseInstance()
        # pygame.quit()
        # break

In [9]:
from ISP_utils import process_HDR

## SHUTTER_TIME(us) = 6667 # ex) 11ms = 11,000us
# shutter_time = [440*1000, 440*4*1000, 440*16*1000, 440*64*1000]
shutter_time = [440*1000, 440*4*1000, 440*16*1000]
# save_dir = 'C:/Users/owner/Desktop/20240607_test_0'
save_dir = 'C:/Users/owner/Desktop/20240704_scene05'
# save_dir = 'C:/Users/owner/Desktop/20240704_black'

for i in shutter_time:
    main(i, save_dir, 'C:/Users/owner/Desktop/patterns_20230522/', OLAT=True)
    
process_HDR(save_dir+'/main', shutter_time)     
# process_HDR(save_dir+'/side', shutter_time)     
# process_RAW(save_dir+'/main')

Library version: 2.6.0.156
<PySpin.PySpin.CameraList; proxy of <Swig Object of type 'Spinnaker::CameraList *' at 0x0000022D5C3F1CC0> >
Number of cameras detected: 2
Blackfly S BFS-U3-51S5PC
24049908
Pixel format not available...
WB blue set to 1.0...
WB red set to 1.0...
Shutter time set to 440.0 ms...

Trigger mode disabled...
Trigger selector set to frame start...
Trigger source set to software...
Trigger mode turned back on...
Acquisition mode set to continuous...
balance ratio: 1.0 exposure time 440001.0
Stream Buffer Count Mode set to manual...

Default Buffer Handling Mode: Oldest First
Default Buffer Count: 10
Maximum Buffer Count: 103
Buffer count now set to: 3


Buffer Handling Mode has been set to Oldest First
Blackfly S BFS-U3-51S5PC
20356250
Pixel format not available...
WB blue set to 1.0...
WB red set to 1.0...
Shutter time set to 440.0 ms...

Trigger mode disabled...
Trigger selector set to frame start...
Trigger source set to software...
Trigger mode turned back on...
A

In [1]:
from ISP_utils import process_HDR

shutter_time = [440*1000, 440*4*1000, 440*16*1000]

# save_dir = 'C:/Users/owner/Desktop/20240703_scene02'
save_dir = 'C:/Users/owner/Desktop/20240704_scene03'

process_HDR(save_dir+'/main', shutter_time)

['000_440000.png', '001_440000.png', '002_440000.png', '003_440000.png', '004_440000.png', '005_440000.png', '006_440000.png', '007_440000.png', '008_440000.png', '009_440000.png', '010_440000.png', '011_440000.png', '012_440000.png', '013_440000.png', '014_440000.png', '015_440000.png', '016_440000.png', '017_440000.png', '018_440000.png', '019_440000.png', '020_440000.png', '021_440000.png', '022_440000.png', '023_440000.png', '024_440000.png', '025_440000.png', '026_440000.png', '027_440000.png', '028_440000.png', '029_440000.png', '030_440000.png', '031_440000.png', '032_440000.png', '033_440000.png', '034_440000.png', '035_440000.png', '036_440000.png', '037_440000.png', '038_440000.png', '039_440000.png', '040_440000.png', '041_440000.png', '042_440000.png', '043_440000.png', '044_440000.png', '045_440000.png', '046_440000.png', '047_440000.png', '048_440000.png', '049_440000.png', '050_440000.png', '051_440000.png', '052_440000.png', '053_440000.png', '054_440000.png', '055_4400

In [2]:
from ISP_utils import process_HDR
shutter_time = [100*1000, 400000, 800000, 1200000]
# shutter_time = [40*4*1000, 40*16*1000, 40*64*1000, 40*64*1000*4]
save_dir = 'C:/Users/owner/Desktop/20240606_glossy'
# save_dir = 'C:/Users/owner/Desktop/20240607_test4'

process_HDR(save_dir+'/main', shutter_time)     
# process_HDR(save_dir+'/side', shutter_time)     

['000_100000.png', '001_100000.png', '002_100000.png', '003_100000.png', '004_100000.png', '005_100000.png', '006_100000.png', '007_100000.png', '008_100000.png', '009_100000.png', '010_100000.png', '011_100000.png', '012_100000.png', '013_100000.png', '014_100000.png', '015_100000.png', '016_100000.png', '017_100000.png', '018_100000.png', '019_100000.png', '020_100000.png', '021_100000.png', '022_100000.png', '023_100000.png', '024_100000.png', '025_100000.png', '026_100000.png', '027_100000.png', '028_100000.png', '029_100000.png', '030_100000.png', '031_100000.png', '032_100000.png', '033_100000.png', '034_100000.png', '035_100000.png', '036_100000.png', '037_100000.png', '038_100000.png', '039_100000.png', '040_100000.png', '041_100000.png', '042_100000.png', '043_100000.png', '044_100000.png', '045_100000.png', '046_100000.png', '047_100000.png', '048_100000.png', '049_100000.png', '050_100000.png', '051_100000.png', '052_100000.png', '053_100000.png', '054_100000.png', '055_1000